<a href="https://colab.research.google.com/github/addo561/learning-pytorch/blob/main/Alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
import  torchvision.transforms as transforms
!pip install torchinfo
from torchinfo  import summary

In [8]:
transforms =  transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [29]:
class Alexnet(nn.Module):
  def __init__(self):
    super().__init__()
    self.features = nn.Sequential(
        nn.Conv2d(3,96,kernel_size=11,stride=4),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3,stride=2),

        nn.Conv2d(96,256,kernel_size=5,padding=2), #paddign = kernel_size//2 ,output shape (w-k+2p/stride) + 1, w = input_size,k=kernel_size,p=padding
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3,stride=2),

        nn.Conv2d(256,384,kernel_size=3,padding=1),
        nn.ReLU(),

        nn.Conv2d(384,384,kernel_size=3,padding=1),
        nn.ReLU(),

        nn.Conv2d(384,256,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3,stride=2)

    )
    with torch.no_grad():
      dummy = torch.randn(1,3,224,224)
      out = self.features(dummy)
      flattened =  out.view(1,-1).size(1)  #  [1,256,5,5] to [1,256*5*5]

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(flattened,4096),
        nn.ReLU(),
        nn.Dropout(0.5),

        nn.Linear(4096,4096),
        nn.ReLU(),
        nn.Dropout(0.5),

        nn.Linear(4096,1000),
    )

  def  forward(self,x):
    x  = self.features(x)
    #x = x.view(x.size(0),-1) Fully connected layers expect 2D input: (batch_size, features)
    x  =  self.classifier(x)
    return x

In [30]:
model = Alexnet()
model

Alexnet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=6400, out_features=4096, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=4096, out_features=4096, bias=True)
    (5

In [2]:
!pip install --upgrade torchvision

In [31]:
summary(model,(1,3,224,224))

Layer (type:depth-idx)                   Output Shape              Param #
Alexnet                                  [1, 1000]                 --
├─Sequential: 1-1                        [1, 256, 5, 5]            --
│    └─Conv2d: 2-1                       [1, 96, 54, 54]           34,944
│    └─ReLU: 2-2                         [1, 96, 54, 54]           --
│    └─MaxPool2d: 2-3                    [1, 96, 26, 26]           --
│    └─Conv2d: 2-4                       [1, 256, 26, 26]          614,656
│    └─ReLU: 2-5                         [1, 256, 26, 26]          --
│    └─MaxPool2d: 2-6                    [1, 256, 12, 12]          --
│    └─Conv2d: 2-7                       [1, 384, 12, 12]          885,120
│    └─ReLU: 2-8                         [1, 384, 12, 12]          --
│    └─Conv2d: 2-9                       [1, 384, 12, 12]          1,327,488
│    └─ReLU: 2-10                        [1, 384, 12, 12]          --
│    └─Conv2d: 2-11                      [1, 256, 12, 12]       

In [21]:
model_w = models.AlexNet_Weights.IMAGENET1K_V1
modelP = models.alexnet(weights=model_w)


In [27]:
summary(modelP,(1,3,224,224))  #  Alexnet single  gpu from pytorch

Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [1, 1000]                 --
├─Sequential: 1-1                        [1, 256, 6, 6]            --
│    └─Conv2d: 2-1                       [1, 64, 55, 55]           23,296
│    └─ReLU: 2-2                         [1, 64, 55, 55]           --
│    └─MaxPool2d: 2-3                    [1, 64, 27, 27]           --
│    └─Conv2d: 2-4                       [1, 192, 27, 27]          307,392
│    └─ReLU: 2-5                         [1, 192, 27, 27]          --
│    └─MaxPool2d: 2-6                    [1, 192, 13, 13]          --
│    └─Conv2d: 2-7                       [1, 384, 13, 13]          663,936
│    └─ReLU: 2-8                         [1, 384, 13, 13]          --
│    └─Conv2d: 2-9                       [1, 256, 13, 13]          884,992
│    └─ReLU: 2-10                        [1, 256, 13, 13]          --
│    └─Conv2d: 2-11                      [1, 256, 13, 13]         

In [32]:
modelP

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 